<a href="https://colab.research.google.com/github/mohamedelgawish511-bot/ai-platform-assigments/blob/main/assigment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import time

def run_keras_model():
    """
    Builds, trains, and evaluates a CNN model using Keras/TensorFlow
    on the Fashion MNIST dataset.
    """
    print("\n" + "="*50)
    print("      STARTING KERAS/TENSORFLOW CNN MODEL")
    print("="*50)

    # Set TensorFlow log level to suppress warnings
    tf.get_logger().setLevel('ERROR')

    # 1. Load the Fashion MNIST dataset
    print("1. Loading and preparing Keras data...")
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

    # 2. Preprocess the data

    # Normalize pixel values to be between 0 and 1
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0

    # CNN requires an explicit channel dimension: (28, 28) -> (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    print(f"Keras Training data shape: {x_train.shape}")

    # 3. Define the CNN Model Architecture
    model = Sequential([
        # Input shape required only for the first layer
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),

        Flatten(),

        Dropout(0.5),
        Dense(100, activation='relu'),

        # Output layer with 10 classes (Fashion MNIST)
        Dense(10, activation='softmax')
    ])

    # 4. Compile the Model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Optional: Display model summary
    # model.summary()

    # 5. Train the Model
    print("\n2. Starting Keras model training (5 epochs)...")
    start_time = time.time()
    model.fit(x_train, y_train, epochs=5, batch_size=64,
                        validation_data=(x_test, y_test), verbose=1)
    end_time = time.time()
    print(f"\nKeras Training Time: {end_time - start_time:.2f} seconds.")


    # 6. Evaluate the Model
    print("\n3. Evaluating Keras model performance...")
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"\n[Keras Result] Test Accuracy: {acc*100:.2f}%")
    print(f"[Keras Result] Test Loss: {loss:.4f}")
    print("="*50)

# =================================================================
# CNN Model 2: PyTorch Implementation
# =================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define the CNN Architecture (nn.Module subclass)
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()

        # Conv Block 1: 1->32 channels, 28x28 -> 14x14
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Conv Block 2: 32->64 channels, 14x14 -> 6x6
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3), # Output 12x12
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)                     # Output 6x6
        )

        # Fully Connected Layers: 64*6*6 = 2304
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(in_features=64 * 6 * 6, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# 2. Setup Data Loading
def load_fashion_mnist_data(batch_size=64):
    """Loads and preprocesses Fashion MNIST data using PyTorch DataLoader."""
    # Common transformation for PyTorch CNNs: ToTensor and normalization
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# 3. Training Loop
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    """Trains the PyTorch model."""
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

# 4. Evaluation Loop
def evaluate_model(model, test_loader):
    """Evaluates the PyTorch model on the test dataset."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def run_pytorch_model():
    """Main function to setup and run the PyTorch CNN."""
    print("\n" + "="*50)
    print("       STARTING PYTORCH CNN MODEL")
    print("="*50)
    print(f"Using device: {device}")

    # Configuration
    epochs = 5
    batch_size = 64
    learning_rate = 0.001

    # Load data
    print("1. Loading and preparing PyTorch data...")
    train_loader, test_loader = load_fashion_mnist_data(batch_size)

    # Instantiate model, loss, and optimizer
    model = FashionCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train
    print("\n2. Starting PyTorch model training (5 epochs)...")
    start_time = time.time()
    train_model(model, train_loader, criterion, optimizer, epochs)
    end_time = time.time()
    print(f"PyTorch Training Time: {end_time - start_time:.2f} seconds.")

    # Evaluate
    print("\n3. Evaluating PyTorch model performance...")
    accuracy = evaluate_model(model, test_loader)
    print(f"\n[PyTorch Result] Test Accuracy: {accuracy:.2f}%")
    print("="*50)

# =================================================================
# Execute Both Models
# =================================================================
if __name__ == '__main__':
    run_keras_model()
    run_pytorch_model()


      STARTING KERAS/TENSORFLOW CNN MODEL
1. Loading and preparing Keras data...
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Keras Training data shape: (60000, 28, 28, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



2. Starting Keras model training (5 epochs)...
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.6860 - loss: 0.8609 - val_accuracy: 0.8544 - val_loss: 0.4060
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.8467 - loss: 0.4212 - val_accuracy: 0.8784 - val_loss: 0.3381
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8706 - loss: 0.3545 - val_accuracy: 0.8770 - val_loss: 0.3319
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8854 - loss: 0.3131 - val_accuracy: 0.8934 - val_loss: 0.2975
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8896 - loss: 0.2977 - val_accuracy: 0.8959 - val_loss: 0.2822

Keras Training Time: 36.81 seconds.

3. Evaluating Keras model performance...

[Keras Result] Test Accuracy: 89.59%
[Keras Result] Test Loss: 0.2822

       STARTING PYTORCH CNN MODEL
Using device: cuda
1. Loading and preparing PyTorch data...


100%|██████████| 26.4M/26.4M [00:02<00:00, 11.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 209kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.92MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.4MB/s]



2. Starting PyTorch model training (5 epochs)...
Epoch 1/5, Loss: 0.4637
Epoch 2/5, Loss: 0.3389
Epoch 3/5, Loss: 0.3074
Epoch 4/5, Loss: 0.2869
Epoch 5/5, Loss: 0.2722
PyTorch Training Time: 75.20 seconds.

3. Evaluating PyTorch model performance...

[PyTorch Result] Test Accuracy: 90.23%
